In [3]:
using Unitful #https://painterqubits.github.io/Unitful.jl/stable/
#quantity * @u_str("unit abbreviation") 
using Symbolics #https://symbolics.juliasymbolics.org/dev/
#cite https://doi.org/10.48550/arXiv.2105.03949
using Latexify
using Test
#1 * @u_str("mA") is 1 milliamp
using CSV, DataFrames
#using Plots
using PlotlyJS
using Printf
using SymPy
using PDFIO
using Unzip
using Interpolations
using Plots
using TensorCast
using CatViews
using MappedArrays
#plotlyjs()
data_dir = "C:\\Cross-Section-Data\\EXFOR\\"

"C:\\Cross-Section-Data\\EXFOR\\"

#Design

    Looping check_line() through each line in the file detects where each dataset begins. 

    Reading the line above each output of check_line() gives the number of rows in each dataset.

    make_spacing_dict() gets the names of the columns and the instructions for how to read each dataset.

    read_dataset() reads each and constructs a dictionary from each dataset.
    
    read_exfor_file() runs all of the above in order to return a single DataFrame of all the data stored at a given file path.
     

In [4]:
function check_line(line, start)
    if length(line) > length(start) - 1
        return (line[1:length(start)] == start)
    end
    return false
end

function make_spacing_dict(line1, line2)
    spacing_ends = [collect(out)[1] for out in findall(">", line2)]
    spacing_starts = append!([1], [collect(out)[1] for out in findall("<", line2)])
    #Find where some of the spacings begin and end based on the arrows
    if length(spacing_ends) != length(spacing_starts)
        print("Error: improper formatting")
        return 
    end
    indices_with_gaps = [index for index in  1:length(spacing_ends)-1 if 
                    spacing_ends[index] != spacing_starts[index+1]-1]
    #Some of the spacings are instead denoted by the letter o instead of arrows
    missing_spacings = [spacing_ends[index]+1:spacing_starts[index+1]-1 for index in indices_with_gaps]
    spacings = append!(missing_spacings, 
    [spacing_starts[i]:spacing_ends[i] for i in 1:length(spacing_ends)])
    spacing_dict = Dict([])
    #Make a dictionary where the keys are name for each column and the values are the indices of the columns
    spacing_names = [spacing_dict[strip(line1[spacing], [' ', '#'])] = spacing for spacing in spacings]
    return spacing_dict
end

function read_datum(datum)
    #Reads a single datum from a line of data
    datum = strip(datum, [' '])
    out = tryparse(Float64, datum)
    if out == nothing
        return datum 
    end
    return out
end

read_datum (generic function with 1 method)

In [5]:
function check_spacings(file_path)
    #Reads an EXFOR file and returns a dictionary of data
    file_as_vector = readlines(file_path)
    spacing_specifiers = [index for index in 1:length(file_as_vector) 
                        if check_line(file_as_vector[index], "# Prj")]
    if spacing_specifiers == []
        print("Error: no data found")
        return
    end
    list_spacing_dict = [make_spacing_dict(file_as_vector[spacing_specifier], 
                                            file_as_vector[spacing_specifier+1])
                        for spacing_specifier in spacing_specifiers]
    #check if the spacing dicts are the same
    for index in 2:length(spacing_specifiers)
        if list_spacing_dict[index] != list_spacing_dict[1]
            return false
        end
    end
    return true
end

check_spacings (generic function with 1 method)

In [6]:
subdirs = [content * "\\" for content in readdir(data_dir) if isdir(data_dir * "\\" * content)]
for subdir in subdirs
    files = readdir(data_dir * subdir)
    test = [check_spacings(data_dir * subdir * file) for file in files]
    println("It is ", all(test),
     " that the spacings in each file are all self consistent for the subdirectory ", subdir)
end

It is true that the spacings in each file are all self consistent for the subdirectory alphas\
It is true that the spacings in each file are all self consistent for the subdirectory deuterons\
It is true that the spacings in each file are all self consistent for the subdirectory gammas\
It is true that the spacings in each file are all self consistent for the subdirectory helions\
It is true that the spacings in each file are all self consistent for the subdirectory neutrons\
It is true that the spacings in each file are all self consistent for the subdirectory other\
It is true that the spacings in each file are all self consistent for the subdirectory protons\


So I only need to retrieve the spacings of the data once for each file. 

In [32]:
function read_exfor_file(file_path, MTs)
    #Reads an EXFOR file and returns a dictionary of data
    file_as_vector = readlines(file_path)
    spacing_specifiers = [index for index in 1:length(file_as_vector) 
                            if check_line(file_as_vector[index], "# Prj") && 
                                tryparse(Int64, file_as_vector[index-5][5:end]) 
                                in MTs]
    if spacing_specifiers == []
        print("Error: no data found")
        return
    end
    dataset_rows = [tryparse(Int64, split(file_as_vector[index - 1], [' '])[end])
                            for index in spacing_specifiers]
    #Each dataset should have the same column names
    spacing_dict = make_spacing_dict(file_as_vector[spacing_specifiers[1]], 
                    file_as_vector[spacing_specifiers[1] + 1])
    #make an empty dataframe to fill up with the data
    df = DataFrame([Vector{Union{Missing, Float64, String, SubString{String}}}(
                missing, sum(dataset_rows)) 
                   for _ in 1:length(keys(spacing_dict))], 
                   [key for key in keys(spacing_dict)])
    #make a nested vector, then flatten it to populate the dataframe.
    _ = [df[:,key] = collect(Iterators.flatten([[read_datum(
                    file_as_vector[line_num][spacing_dict[key]]) 
                    for line_num in spacing_specifiers[index] + 2: 
                            spacing_specifiers[index] + 1 + dataset_rows[index]]
                            for index in 1:length(spacing_specifiers)]))
                    for key in keys(spacing_dict)]           
    return df
end

read_exfor_file (generic function with 1 method)

Retrieving all of data at the correct spacings (without reorganizing it) takes 4 seconds for a large file. 

In [33]:
file_path = data_dir * "neutrons\\092_U_238.c4"
a = read_exfor_file(file_path, [102])

,Targ,Entry,dCos/LO,I78,dEnergy,dELV/HL,M,Refer (YY)
,Union…?,Union…?,Union…?,Union…?,Union…?,Union…?,Union…?,Union…?
1,92238.0,22115.0,,,,,,"F.Corvi,ET.AL. (88)"
2,92238.0,22115.0,,,,,,"F.Corvi,ET.AL. (88)"
3,92238.0,22115.0,,,,,,"F.Corvi,ET.AL. (88)"
4,92238.0,22115.0,,,,,,"F.Corvi,ET.AL. (88)"
5,92238.0,22115.0,,,,,,"F.Corvi,ET.AL. (88)"
6,92238.0,22115.0,,,,,,"F.Corvi,ET.AL. (88)"
7,92238.0,22115.0,,,,,,"F.Corvi,ET.AL. (88)"
8,92238.0,22115.0,,,,,,"F.Corvi,ET.AL. (88)"
9,92238.0,22115.0,,,,,,"F.Corvi,ET.AL. (88)"


In [8]:
names(a)

18-element Vector{String}:
 "Targ"
 "Entry"
 "dCos/LO"
 "I78"
 "dEnergy"
 "dELV/HL"
 "M"
 "Refer (YY)"
 "Prj"
 "Energy"
 "Cos/LO"
 "ELV/HL"
 "MF"
 "dData"
 "Data"
 "Sub"
 "PXC"
 "MT"

In [9]:
unique(a[!,"MT"])

29-element Vector{Union{Missing, Float64, SubString{String}, String}}:
  452.0
  455.0
  456.0
    1.0
    2.0
    3.0
    4.0
   16.0
   17.0
   18.0
    ⋮
 6002.0
 6003.0
 6004.0
 6010.0
 6021.0
 6022.0
 6023.0
 6024.0
 6031.0